In [1]:
import astropy
from astropy.io import fits 
from astropy.time import Time
from astropy.visualization import time_support
from astropy.timeseries import LombScargle
from astropy.convolution import Box1DKernel
from astropy.convolution import convolve
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd
import scipy.signal as sig
from astropy.stats import sigma_clip
import warnings

In [2]:
from functools import partial

import matplotlib as mpl
from scipy import signal, stats
from scipy.ndimage import center_of_mass

# install moonbow first. in repo root:
# pip install -e . --no-deps
from moonbow.fit import Fit

rng = np.random.default_rng()

In [3]:
def dataClean(filename,distance_pc,eff_width): 
    
    """ This function removes nan values and corrects time values for TESS 20 second cadence data. 
        Also corrects TESS flux to energy in ergs."""
    
    #Create an array of time and flux data with nans removed    
    with fits.open(filename, mode="readonly") as hdulist:
        raw_time = hdulist[1].data['TIME']
        raw_flux = hdulist[1].data['PDCSAP_FLUX']
        raw_err = hdulist[1].data['PDCSAP_FLUX_ERR']
    data = np.vstack((raw_time, raw_flux, raw_err))
    nonan_data = data[:, ~np.isnan(data).any(axis=0)]

    times = nonan_data[0]
    flux = nonan_data[1]
    error = nonan_data[2]
    
    #Create Dataframe of cleaned data
    time = np.array(times)
    flux = np.array(flux)
    err = np.array(error)
   

    #Convert energy to ergs
    dist_cm = distance_pc * 3.086e+18
    ergs = []
    ergs_err = []
    
    for i in flux: 
        fluence = i * eff_width
        energy = (4 * np.pi * (dist_cm**2) * fluence)
        ergs.append(energy*1.95829e-9) #AB system zero point
        
    for i in err:
        fluence_err = i*eff_width
        energy_err = (4 * np.pi * (dist_cm**2) * fluence_err)
        ergs_err.append(energy_err*1.95829e-9) #AB system zero point
        
    energy = np.asarray(ergs)
    error = np.asarray(ergs_err)

    
    #Return cleaned data
    return pd.DataFrame({
                        'Time': time,
                        'Energy': energy,
                        'Energy Error': error,
                        'Flux': flux,
                        'Flux Error': err
    })

In [4]:
#Using only until large flare function is completed, need more stars to get a better idea for a good
#model for stellar quiescence generation

def generate_Qcurve(clean_dataframe):
    """ This function takes cleaned data to create a dataframe of the estimated quiescent light curve"""

    q_time = clean_dataframe['Time'] # time associated w/ quiescent flux
    
    #Set and index variables
    fluxes = clean_dataframe['Energy']
    times = clean_dataframe['Time']
    F_err = clean_dataframe['Energy Error'] 

    #Smooth
    smooth_1 = sig.savgol_filter(fluxes,1400,3)
    q_flux = sig.savgol_filter(smooth_1,2000,3)
    
    #Create/return dataframe
    
    return pd.DataFrame({
                'Time': q_time,
                'Quiescent Energy':q_flux
    })


In [5]:
def find_ix_ranges(ix, buffer=False):
    """ Finds indexes in the range.
    
        From MC GALEX function defs"""
    
    foo, bar = [], []
    for n, i in enumerate(ix):
        if len(bar) == 0 or bar[-1] == i-1:
            bar += [i]
        else:
            if buffer:
                bar.append(min(bar)-1)
                bar.append(max(bar)+1)
            foo += [np.sort(bar).tolist()]
            bar = [i]
        if n == len(ix)-1:
            if buffer:
                bar.append(min(bar)-1)
                bar.append(max(bar)+1)
            foo += [np.sort(bar).tolist()]
    return foo

In [6]:
def get_inff(curve,distance_pc, clipsigma=3, quiet=True, band='NUV',
             binsize=20.):
    """ Calculates the Instantaneous Non-Flare Flux values.
    
        From MC GALEX function defs"""
    
    sclip = sigma_clip(np.array(curve['Energy']), sigma_lower=clipsigma, sigma_upper=clipsigma)
    inff = np.median(sclip)
    inff_err = np.sqrt(inff*len(sclip)*binsize)/(len(sclip)*binsize)
    if inff and not quiet:
        print('Quiescent at {m} AB mag.'.format(m=gt.counts2mag(inff, band)))
    inff_ = ((inff*(3.898e-5))*(4*np.pi*((distance_pc*3.086e+18)**2)))*1.95829e-9
    inff_er = ((inff_err*(3.898e-5))*(4*np.pi*((distance_pc*3.086e+18)**2)))*1.95829e-9
    return inff, inff_er

In [7]:
def find_flare_ranges(curve,q_curve,sig,dist,quiescence=None):
    """This function will run through the data to find 
    flares and ranges of flares. This function will return
    a table of flares ranges. 
    
    Adapted from MC GALEX function defs.
    Stage: complete"""
    
    tranges = [[min(curve['Time']), max(curve['Time'])]] 
    if not quiescence:
        q, q_err = get_inff(curve,dist)
    else:
        q, q_err = quiescence
    flare_ranges = []
    for trange in tranges:
        ix = np.where(((np.array(curve['Energy'].values)-(sig*np.array(curve['Energy Error'].values)) >= q_curve['Quiescent Energy'])))[0]
        flareFlux = ix
        if not len(ix):
            continue
        flux_ix = []
        
        for ix_range in find_ix_ranges(ix):
            # go backwards
            consec = 0 
            err = curve.iloc[ix_range[0]]['Energy Error'] 
            
            #while flux - err > quiescence, find 2 consecutive points within quiescent curve
            while (curve.iloc[ix_range[0]]['Energy']-err >= q_curve.iloc[ix_range[0]]['Quiescent Energy']) and ix_range[0] > 0 or (consec < 1 and ix_range[0] >0):
                err = curve.iloc[ix_range[0]]['Energy Error']

                if curve.iloc[ix_range[0]]['Energy']- err < q_curve.iloc[ix_range[0]]['Quiescent Energy']:
                    consec +=1
                else: 
                    consec = 0
                if (curve.iloc[ix_range[0]+1]['Time']-curve.iloc[ix_range[0]]['Time']) > 1000: 
                    break               
                ix_range = [ix_range[0] - 1] + ix_range
                
                # go forwards
            consec = 0 
            err = curve.iloc[ix_range[-1]]['Energy Error']
            while (curve.iloc[ix_range[-1]]['Energy']-err >= q_curve.iloc[ix_range[-1]]['Quiescent Energy']) and ix_range[-1] != len(curve)-1 or (consec <1 and ix_range[:-1]!= len(curve)-1):
                err = curve.iloc[ix_range[-1]]['Energy Error']
                if curve.iloc[ix_range[-1]]['Energy']-err < q_curve.iloc[ix_range[-1]]['Quiescent Energy']:
                    consec += 1
                else: 
                    consec = 0
                if curve.iloc[ix_range[0]]['Time']-curve.iloc[ix_range[-1]]['Time'] > 1000: 
                    break
                ix_range = ix_range + [ix_range[-1] + 1]
                
            flux_ix += ix_range
        ix = np.unique(flux_ix)
        flare_ranges += find_ix_ranges(list(np.array(ix).flatten()))
    return(flare_ranges,flareFlux)


In [8]:
def refine_flare_ranges(curve,q_curve, sig=3., flare_ranges=None):
    """ Identify the start and stop indexes of a flare event after
    refining the INFF by masking out the initial flare detection indexes. 
    From MC GALEX function defs."""
    time_support()
    if not flare_ranges:
        flare_ranges, _ = find_flare_ranges(curve, q_curve, sig)
    flare_ix = list(itertools.chain.from_iterable(flare_ranges))
    quiescience_mask = [False if i in flare_ix else True for i in
                        np.arange(len(curve['Time']))]
    quiescence = q_curve
    quiescence_err = (np.sqrt(curve['Energy'][quiescience_mask].sum())/curve['Energy'].sum())
    flare_ranges, flare_3sigs = find_flare_ranges(curve,q_curve,
                                                  quiescence=(quiescence,
                                                              quiescence_err),
                                                  sig=sig)
    flare_ix = list(itertools.chain.from_iterable(flare_ranges))
    not_flare_ix = list(set([x for x in range(len(curve['Time']))]) - set(flare_ix))
      
    return flare_ranges, flare_ix

In [ ]:
def qpp_find(flaring_frame,ranges_ix,energy_ix):
    
    ranges = []
    for i,v in enumerate(ranges_ix):
        t = flaring_frame['Time'][v]
        f = flaring_frame['Flux'][v]
        ranges.append([i,t,f])

        #Select Large Flares

    large_flares = []
    for i,v in enumerate(ranges_ix):
        if len(v)>=45:                    #~15 minute minimum                    
            large_flares.append((v))
        else:
            continue

#Match Data to Large Flares

    large_flare_data = []
    for i,v in enumerate(large_flares):
        data = flaring_frame['Energy'][large_flares[i]]
        large_flare_data.append(data)
    
    norm_flares=[] #normalize flare values
    for i in range(len(large_flare_data)):
        mx = max(large_flare_data[i])
        norm_flares.append(large_flare_data[i]/mx)
    
    times=[]
    for i in range(len(large_flares)):
        t = flaring_frame['Time'][large_flare_data[i].index]
        times.append(t)
        
    lg_flare_data=[]
    lg_flare_time=[]
    lg_flare_det=[]
    lg_flare_params = []
    flare_fits = []

    for i in range(len(large_flares)):
        model = aflare2
        x = flaring_frame['Time'][large_flares[i]]
        y = norm_flares[i]
        data = pd.DataFrame({'time': x, 'energy': y})
    
        rise = x[0:np.argmax(y)].values
        fwhm = (max(rise)-min(rise))*2
    
        fitter = Fit(
        underlying_function=model,
        dimensionality=1,
        data=data,
        dependent_variable='energy')
    
        fitter.make_vector(independent_variables=['time'])
        fitter.fit(guess=np.array((max(rise),fwhm,1,
                              1.00000, 1.94053, -0.175084, -2.3769, -1.12498,
                              0.689008, -1.600536, 0.302963, -0.278318))) 
        if fitter.det > 0.70:
            lg_flare_time.append(x)
            lg_flare_data.append(y)
            lg_flare_det.append(fitter.det)
            fit_params, covariance_matrix = fitter.curve_fit
            lg_flare_params.append((fit_params))
            fit = fitter.fitted_curve 
            flare_fits.append((fit))
    return lg_flare_data, lg_flare_time, lg_flare_det, lg_flare_params, flare_fits

In [61]:
def fit_residuals(lg_flare_data,flare_fits):
    residuals = []
    for i in range(len(lg_flare_data)):
        res_i = lg_flare_data[i]-flare_fits[i]
        residuals.append((res_i))

    kernel = Box1DKernel(8)
    res_fits = []
    for i in range(len(residuals)):
        res_fit = sig.savgol_filter(residuals[i],21,3)
        fit2 = convolve(res_fit,kernel,boundary='extend')
        res_fits.append(fit2)
    
    return(residuals, res_fits)


#to view/plot residual fits: 
    
#fig,ax = plt.subplots((len(lg_flare_data)))
#for i in range(len(res_fits)):
#    ax.flat[i].plot(range(len(residuals[i])),residuals[i],c='grey')
#    ax.flat[i].plot(range(len(res_fits[i])),res_fits[i],c='black')

In [62]:
def aflare2(t, tpeak, fwhm, ampl,
            c0,c1,c2,c3,c4,
            d0,d1,d2,d3):
#    t : 1-d array
#        The time array to evaluate the flare over
#    tpeak : float
#        The time of the flare peak
#    fwhm : float
#        The "Full Width at Half Maximum", timescale of the flare
#   ampl : float
#        The amplitude of the flare
#    Returns
#    flare : 1-d array
#        The flux of the flare model evaluated at each time
    #_fr = [1.00000, 1.94053, -0.175084, -2.24588, -1.12498]
    _fr = [c0,c1,c2,c3,c4]
    #_fd = [0.689008, -1.60053, 0.302963, -0.278318]
    _fd = [d0,d1,d2,d3]
    
    t = np.array(t)
    flare = np.piecewise(t, [(t<= tpeak) * (t-tpeak)/fwhm > -1.,
                                (t > tpeak)],
                            [lambda x: (_fr[0]+                       # 0th order
                                        _fr[1]*((x-tpeak)/fwhm)+      # 1st order
                                        _fr[2]*((x-tpeak)/fwhm)**2.+  # 2nd order
                                        _fr[3]*((x-tpeak)/fwhm)**3.+  # 3rd order
                                        _fr[4]*((x-tpeak)/fwhm)**4. ),# 4th order
                             lambda x: (_fd[0]*np.exp( ((x-tpeak)/fwhm)*_fd[1] ) +
                                        _fd[2]*np.exp( ((x-tpeak)/fwhm)*_fd[3] ))]
                            ) * np.abs(ampl) # amplitude

    return flare

In [63]:
rawdata = '/Users/katborski/Documents/GitHub/AFPSC/TESS/tess2021232031932-s0042-0000000250081915-0213-a_fast-lc.fits'
flaring_frame = dataClean(rawdata,35,3.898e-5) #35=dist in pc, 3.898e-5=eff width for Af Psc
q_frame = generate_Qcurve(flaring_frame)

In [65]:
ranges_ix,energy_ix = find_flare_ranges(flaring_frame, q_frame, 3,35, quiescence=None)

In [66]:
qpps = qpp_find(flaring_frame, ranges_ix, energy_ix)

In [67]:
res_fit = fit_residuals(qpps[0],qpps[4])

In [70]:
#to view/plot davenport fit: 

#fig,ax = plt.subplots((len(lg_flare_data)))
#for i in range(len(lg_flare_data)):    
#    ax.flat[i].plot(lg_flare_time[i],lg_flare_data[i])
#    ax.flat[i].plot(lg_flare_time[i],fits_[i],c='black')
#    ax.flat[i].set_title(lg_flare_det[i])
#    plt.setp(ax, xticks=[]) 

In [71]:
#to view/plot residual fits: 
    
#fig,ax = plt.subplots((len(lg_flare_data)))
#for i in range(len(res_fits)):
#    ax.flat[i].plot(range(len(residuals[i])),residuals[i],c='grey')
#    ax.flat[i].plot(range(len(res_fits[i])),res_fits[i],c='black')

### TEST with EV Lac

In [72]:
test_rawdata = '/Users/katborski/Documents/GitHub/QPPs/moonbow-main/tess2022244194134-s0056-0000000154101678-0243-a_fast-lc.fits'
test_flaring_frame = dataClean(test_rawdata,5.05,3.898e-5) #5.05=dist in pc to EV Lac, 3.898e-5=eff width for TESS
test_q_frame = generate_Qcurve(test_flaring_frame)


In [73]:
test_ranges_ix,test_energy_ix = find_flare_ranges(test_flaring_frame,test_q_frame,4,5.05,quiescence=None)

In [74]:
test_qpps = qpp_find(test_flaring_frame, test_ranges_ix, test_energy_ix)

In [75]:
test_res_fit = fit_residuals(test_qpps[0],test_qpps[4])

In [87]:
#to view/plot davenport fit: 

#fig,ax = plt.subplots((len(test_qpps[0])))
#for i in range(len(test_qpps[0])):    
#    ax.flat[i].plot(test_qpps[1][i],test_qpps[0][i])
#    ax.flat[i].plot(test_qpps[1][i],test_qpps[4][i],c='black')
#    ax.flat[i].set_title(test_qpps[2][i])
#    plt.setp(ax, xticks=[]) 
#plt.show()

In [86]:
#to view/plot residual fits: 
    
#fig,ax = plt.subplots((len(test_res_fit[0])))
#for i in range(len(test_res_fit[0])):
#    ax.flat[i].plot(range(len(test_res_fit[0][i])),test_res_fit[0][i],c='grey')
#    ax.flat[i].plot(range(len(test_res_fit[1][i])),test_res_fit[1][i],c='black')
#plt.show()

In [ ]:
#ALL WORKING :)

#run a big load of data through

sigma = # self select

for i in manifest:
    distance_pc = # get dist from file
    flaring_frame = dataclean(manifest[i])
    q_frame = generate_Qcurve(flaring_frame)
    ranges_ix, energy_ix = find_flare_ranges(flaring_frame,q_frame,sigma,distance_pc,quiescence=None)
    qpps = qpp_find(flaring_frame,ranges_ix,energy_ix)
    res_fits = fit_residuals(qpps[0],qpps[4])

In [88]:
#attempt to get more 20sec data from astroquery??

from astroquery.mast import Observations


In [115]:
tess_20sec = Observations.query_criteria(obs_collection = 'TESS', sequence_number = [40], t_exptime = [20], 
                                         dataproduct_type = "timeseries")


In [117]:
test_set = tess_20sec[:99]


In [118]:
dataproducts = Observations.get_product_list(test_set)

In [120]:
manifest = Observations.download_products(dataproducts, productType="SCIENCE",productSubGroupDescription='LC')

INFO: Found cached file ./mastDownload/TESS/tess2021175071901-s0040-0000000021496569-0211-s/tess2021175071901-s0040-0000000021496569-0211-s_lc.fits with expected size 2062080. [astroquery.query]
INFO: Found cached file ./mastDownload/TESS/tess2021175071901-s0040-0000000022999915-0211-s/tess2021175071901-s0040-0000000022999915-0211-s_lc.fits with expected size 2062080. [astroquery.query]
INFO: Found cached file ./mastDownload/TESS/tess2021175071901-s0040-0000000043051232-0211-s/tess2021175071901-s0040-0000000043051232-0211-s_lc.fits with expected size 2062080. [astroquery.query]
INFO: Found cached file ./mastDownload/TESS/tess2021175071901-s0040-0000000047582559-0211-s/tess2021175071901-s0040-0000000047582559-0211-s_lc.fits with expected size 2062080. [astroquery.query]
INFO: Found cached file ./mastDownload/TESS/tess2021175071901-s0040-0000000048424655-0211-s/tess2021175071901-s0040-0000000048424655-0211-s_lc.fits with expected size 2062080. [astroquery.query]
INFO: Found cached file .

In [124]:
manifest

Local Path,Status,Message,URL
str123,str8,object,object
./mastDownload/TESS/tess2021175071901-s0040-0000000021496569-0211-s/tess2021175071901-s0040-0000000021496569-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000000022999915-0211-s/tess2021175071901-s0040-0000000022999915-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000000043051232-0211-s/tess2021175071901-s0040-0000000043051232-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000000047582559-0211-s/tess2021175071901-s0040-0000000047582559-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000000048424655-0211-s/tess2021175071901-s0040-0000000048424655-0211-s_lc.fits,COMPLETE,None,None
...,...,...,...
./mastDownload/TESS/tess2021175071901-s0040-0000001102441603-0211-s/tess2021175071901-s0040-0000001102441603-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000001201355818-0211-s/tess2021175071901-s0040-0000001201355818-0211-s_lc.fits,COMPLETE,None,None
./mastDownload/TESS/tess2021175071901-s0040-0000001714363391-0211-s/tess2021175071901-s0040-0000001714363391-0211-s_lc.fits,COMPLETE,None,None
